# Classifying Images with a DNN Model

## Introduction
In this notebook, we demonstrate how to build an image classification model using a Deep Neural Network (DNN). The model is trained to recognize different types of dishes from images, classified into six categories: *American*, *Chinese*, *European*, *Indian*, *Japanese*, and *Korean*.

## Installing Dependencies
Ensure you are using Python 3.9 or higher. To install all required packages, use the following command:

In [ ]:
!pip install -r requirements.txt

## Required Libraries
We begin by importing all necessary libraries for image loading, model building, training, and visualization.

In [ ]:
from os import path
import tensorflow as tf
from keras import utils, Input, Model
from keras.applications.efficientnet import EfficientNetB0
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from matplotlib import pyplot as plt

## Configuration
We define essential constants for image size, batch size, and the number of output classes.

In [ ]:
NUM_CLASSES = 6
IMAGE_SIZE = 224
IMAGE_CHANNELS = 3
BATCH_SIZE = 64
RANDOM_SEED = 58239
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

## Visualize Sample Images
Let's take a quick look at a few sample images from each class to understand the dataset visually.

In [ ]:
filenames = [
    "dataset/Dishes/American/American_309.jpg", "dataset/Dishes/Chinese/Chinese_751.jpg",
    "dataset/Dishes/European/European_101.jpg", "dataset/Dishes/Indian/Indian_823.jpg",
    "dataset/Dishes/Japanese/Japanese_111.jpg", "dataset/Dishes/Korean/Korean_100.jpg"]
for i, filename in enumerate(filenames):
    plt.subplot(2, 3, i + 1)
    plt.imshow(plt.imread(filename))
    plt.title(f"{path.basename(filename)}")
    plt.axis("off")
plt.show()

## Load and Split Dataset
We use Keras utilities to load images from directories. The folder names are used as labels, and the data is automatically split into training and validation sets using a fixed random seed for reproducibility.

In [ ]:
train_ds = utils.image_dataset_from_directory(
    directory="dataset/dishes/",
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    subset="training",
    seed=RANDOM_SEED,
    validation_split=0.1)

validation_ds = utils.image_dataset_from_directory(
    directory="dataset/dishes/",
    labels="inferred",
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    subset="validation",
    seed=RANDOM_SEED,
    validation_split=0.1)

## Normalize Image Data
We normalize the image pixel values to the range [0, 1] to improve model performance and training stability.

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
validation_ds = validation_ds.map(lambda x, y: (normalization_layer(x), y))

## Build the Model
We employ transfer learning by using EfficientNetB0 as the base model. The top layers are customized for our classification task.

In [ ]:
inputs = Input(shape=INPUT_SHAPE)
base_model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")
base_model.trainable = False

x = GlobalAveragePooling2D(name="avg_pool")(base_model.output)
x = BatchNormalization()(x)
x = Dropout(0.2, name="top_dropout")(x)
outputs = Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

model = Model(inputs, outputs)
model.summary()

## Train the Model
Next, we compile the model using categorical cross-entropy loss and the RMSprop optimizer. The model is then trained on the dataset for 10 epochs.

In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(train_ds, epochs=10, validation_data=validation_ds)

## Evaluate Training Results
We plot the training and validation accuracy over the epochs to assess the model's performance.

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper left")
plt.show()

## Save the Trained Model
After training, we save the model to a file for later use in inference or further training.

In [ ]:
model.save("dishes_model.keras")

🎉 Congratulations! You've successfully built, trained, evaluated, and saved an image classification model using a DNN and transfer learning.